# SPAM DETECTION USING LSTM




In [ ]:
!pip install matplotlib

In [ ]:
from keras.layers import SimpleRNN,GRU,LSTM, Embedding, Dense, Flatten
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard
from keras.utils import plot_model

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np

## Read data

In [ ]:
data = pd.read_csv("./spams.csv")
data1 = pd.read_csv("./spam_text_mails_data.csv")
print(data.tail())
print(data1.head())
print(data1.tail())


In [ ]:
#SEPARATING MAILS AND LABELS(HAM OR SPAM)
mails = []
labels = []
for index, row in data.iterrows():
    mails.append(row['Mail'])
    if row['Category'] == 'ham':
        labels.append(0)
    else:
        labels.append(1)
for index, row in data1.iterrows():
    mails.append(row['Mail'])
    if row['Category'] == 'ham':
        labels.append(0)
    else:
        labels.append(1)
mails = np.asarray(mails)
labels = np.asarray(labels)
   

print("Number of mails: ", len(mails))
print("Number of labels: ", len(labels))


## TOKENIZATION


In [ ]:
max_vocab = 10000
max_len = 500

# Ignore all words except the 10000 most common words
tokenizer = Tokenizer(num_words=max_vocab)
# Calculate the frequency of words
tokenizer.fit_on_texts(mails)
# Convert array of mails to list of sequences of integers
sequences = tokenizer.texts_to_sequences(mails)

# Dict keeping track of words to integer index
word_index = tokenizer.word_index

## PADDING

In [ ]:
# Convert the array of sequences(of integers) to 2D array with padding
# maxlen specifies the maximum length of sequence (truncated if longer, padded if shorter)
data = pad_sequences(sequences, maxlen=max_len)

print("data shape: ", data.shape)


## SEPARATING THE DATA FOR TRAINING AND TESTING

In [ ]:
# We will use 80% of data for training & validation(80% train, 20% validation) and 20% for testing
train_samples = int(len(mails)*0.8)

mails_train = data[:train_samples]
labels_train = labels[:train_samples]

mails_test = data[train_samples:len(mails)-2]
labels_test = labels[train_samples:len(mails)-2]

## CREATING LSTM MODEL

In [ ]:
embedding_mat_columns=32
# Construct the SimpleRNN model
model = Sequential()
    ## Add embedding layer to convert integer encoding to word embeddings(the model learns the
    ## embedding matrix during training), embedding matrix has max_vocab as no. of rows and chosen
    ## no. of columns
model.add(Embedding(input_dim=max_vocab, output_dim=embedding_mat_columns, input_length=max_len))

model.add(LSTM(units=embedding_mat_columns))


model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.summary()

## TRAINING AND TESTING THE MODEL

In [ ]:
 #plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)

    # Training the model
model.fit(mails_train, labels_train, epochs=5, batch_size=60, validation_split=0.2)

    # Testing the model
pred = model.predict_classes(mails_test)
acc = model.evaluate(mails_test, labels_test)
print("Test loss is {0:.2f} accuracy is {1:.2f}  ".format(acc[0],acc[1]))

## TESTING WITH CUSTOM MESSAGE

In [ ]:
# Constructing a custom mail to check model
custom_mail = 'Congratulations you got this mail because you have won a prize of million dollars and world tour'

custom_mail = custom_mail.lower().split(' ')
test_seq = np.array([word_index[word] for word in custom_mail])

test_seq = np.pad(test_seq, (500-len(test_seq), 0), 'constant', constant_values=(0))
test_seq = test_seq.reshape(1, 500)
        

pred = model.predict_classes(test_seq)
print(pred)

In [ ]:
# Constructing a custom mail to check model
custom_mai = 'Congratulations brother for your results in education'
custom_mai = custom_mai.lower().split(' ')
test_seq = np.array([word_index[word] for word in custom_mai])

test_seq = np.pad(test_seq, (500-len(test_seq), 0), 'constant', constant_values=(0))
test_seq = test_seq.reshape(1, 500)
        

pred = model.predict_classes(test_seq)
print(pred)